In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [2]:
# autocorrect, wordprediction
# fibonacci, AP, GP, HP, series and propagation 
# Activity Detection 
# y = f(x, g(y))
# output is transformed and channeled back into input 

# Temp tomorrow-> f(temp_today, temp_day_before.... temp_day_last_year,,, temp_JS, )
# temp_tom = w1*temp_today + w2*history f(temp_relevant_history) + w3*weaker_history.... +wN*weakest history... + 0*the_extreme_history

In [3]:
# BINS -> grouping, Ordering, sorting 
# avg_temp_next_month = strong*(this_month)+ fair_strength*  prev_month + weak*prior_month + (tending_to_0)*history!


In [4]:
# CHARACTER ADDITION-> math equation 
# 4+5=
# 1+2=
# 50,0000

# if user enter 2 numbers: 284+ 16 = ?
# 300

In [5]:
# word prediction
# this is a -> good 
# are you very -> hungry?
# 

In [109]:
HP_trainingsize = 50000
HP_MAX_DIGITS_IN_NUM = 3
HP_MAXLEN = HP_MAX_DIGITS_IN_NUM + 1 + HP_MAX_DIGITS_IN_NUM
HP_OUTPUT_SEQUENCE_LENGTH = 1
HP_BATCH_SIZE = 64
HP_EPOCHS = 15

In [7]:
class corpus:
  def __init__(self, chars): # input a sequence, and build dictionaries
    self.chars = sorted(set(chars)) #Bag of characters
    self.chars.append(' ')
    self.chars.append('+')
    self.word2idx = dict( (word,num) for num,word in enumerate(self.chars) )
    self.idx2word = dict( (num,word) for num,word in enumerate(self.chars) )
    self.charlen = len(self.chars)

  def encoding(self, C, numrows): # encoding logic
    x = np.zeros((numrows, self.charlen ))
    for i,c in enumerate(C):
      x[i, self.word2idx[c]] = 1
    return x

  def decoding(self, x, ismaxlen = True):
    if ismaxlen:
      x = x.argmax(axis=-1) # list of numbers
    return "".join(self.idx2word[xx] for xx in x )


  


In [8]:
# created all possible combinations-> and then eliminated +num and num+, and only keep
# num+num!
# 5+6 = 6+5 
# Reverse my inputs! this is a case where opposite sequence is ALSO the same label!


In [9]:
allchars = "0123456789"
ctable = corpus(allchars)
reverse = True

In [10]:
questions = []
expected = []
# set to check if reverse inputs are also present!
seen = set()
while len(questions) < HP_trainingsize:
  chars_all = list(allchars)
  charlst = lambda: int( "".join(np.random.choice(chars_all) for i in range(np.random.randint(1, HP_MAX_DIGITS_IN_NUM+1))))
  a,b = charlst(), charlst()
  key = tuple(sorted( (a,b)))
  if key in seen:
    continue
  q = "{}+{}".format(a,b)
  seen.add(key)
  # NORMALIZE the length-> PADDING 
  query = q + " " * (HP_MAXLEN - len(q))
  ans = str(a+b)
  if reverse:
    query = query[::-1] 
  questions.append(query)
  expected.append(ans)




In [11]:
len(questions)

50000

In [12]:
len(expected)

50000

In [13]:
print(questions[:10])

[' 971+21', '  72+07', '  7+206', '   7+63', '  77+98', '    4+7', '  763+5', '818+279', '   9+04', '   24+3']


In [14]:
expected[:10]

['191', '97', '609', '43', '166', '11', '372', '1790', '49', '45']

In [15]:
x = np.zeros((len(questions), HP_MAXLEN, len(allchars)+2), dtype=np.bool)
y = np.zeros((len(questions), HP_MAX_DIGITS_IN_NUM +1,len(allchars)+2 ), dtype=np.bool)

In [16]:
999+999 # digit_count(SUM of 2 largest nums) = digital count(their sum) - 1

1998

In [17]:
for i,sentence in enumerate(questions):
  x[i] = ctable.encoding(sentence, HP_MAXLEN)
  y[i] = ctable.encoding(expected[i],  HP_MAX_DIGITS_IN_NUM +1 )
  

In [18]:
indices = np.arange(len(y))
np.random.shuffle(indices)
minorx = x[indices]
minory = y[indices]

partition = int(len(x) * 0.9) # 90% training data
xtrain,xval = x[:partition], x[partition:]
ytrain,yval = y[:partition], y[partition:]

In [19]:
xtrain.shape

(45000, 7, 12)

In [20]:
yval.shape

(5000, 4, 12)

In [54]:
yval[0]

array([[False,  True, False, False, False, False, False, False, False,
        False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False, False, False,
         True, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False]])

In [21]:
indices

array([21960, 21194, 20223, ..., 32443,  6158, 14172])

In [22]:
HP_MAXLEN, len(chars_all)+2 # +2 for ' ' and symbol 


(7, 12)

In [79]:
xtrain[0].shape

(7, 12)

In [88]:
model = keras.Sequential()
layer_lstm = keras.layers.LSTM(128, input_shape=(7, 12)) 
# RepeatVector 
# LSTM-> gave N OUTputs
# the last OUTPUT-> TIME STEP
# 999+999 = 1098
# 99+99 = 198
# max_digit=3, output = 4
# max_digits=2, output = 3
# repeat vector-> store the memory-> the last time vector from previous LSTM
# then => keep repeating-> adding LSTMs 


In [25]:
# 3 words-> 4th onwards! 
# I am going (to)
# am going to (school)
# repeat-> MAXLEN (each word + 1)
# added no. os LSTMs that we wanted to count-> 3

# SUM-> Repeat-> (3) + 1
# LSTMs-> only 1 y! -> 1 

# repeat_vector-> input of size vector/time vector from previous LSTM 
# sequence length LSTM steps


In [89]:
layer_repeated_vector = keras.layers.RepeatVector(4)

In [90]:
# Add all prev layers to model before appending repeat layers
model.add(layer_lstm)
model.add(layer_repeated_vector)
# now add sequence_length no. of LSTMs to learn length pattern 

for i in range(1):
  model.add(keras.layers.LSTM(128, return_sequences=True))


In [91]:
output_dim = 12 # 0-9, ' ' , +
#layer_d1 = keras.layers.Dense(128, activation='relu')
layer_d2 = keras.layers.Dense(output_dim, activation='softmax')

In [92]:
#model.add(layer_d1)
model.add(layer_d2)

In [93]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [94]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_23 (LSTM)               (None, 128)               72192     
_________________________________________________________________
repeat_vector_9 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_24 (LSTM)               (None, 4, 128)            131584    
_________________________________________________________________
dense_7 (Dense)              (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


In [95]:
# RNNs-> keep training and keep observing/validating/visualizing output for errors

# for first time, and then in loop!

print('****')
model.fit(xtrain, ytrain, batch_size=HP_BATCH_SIZE, epochs=1, validation_data=(xval,yval))

****
1407/1407 [==============================] - 17s 10ms/step - loss: 1.7550 - accuracy: 0.2179 - val_loss: 1.8178 - val_accuracy: 0.1342


In [115]:
ind = np.random.randint(0, len(xval))
rowx,rowy = xval[np.array([ind])] , yval[np.array([ind])]
preds = model.predict(rowx)
pred_maxes = np.argmax(preds, axis=-1)

In [116]:
pred_maxes


array([[6, 6, 6, 6]])

In [117]:

np.argmax(rowy, axis=-1)

array([[6, 7, 3, 0]])

In [110]:
for epoch in range(1,3):
  print('****')
  print(epoch)
  model.fit(xtrain, ytrain, batch_size=HP_BATCH_SIZE, epochs=HP_EPOCHS, validation_data=(xval,yval))

****
1
Epoch 1/15
704/704 [==============================] - 7s 10ms/step - loss: 1.7457 - accuracy: 0.2527 - val_loss: 1.8115 - val_accuracy: 0.3224
Epoch 2/15
704/704 [==============================] - 7s 10ms/step - loss: 1.7411 - accuracy: 0.2467 - val_loss: 1.7843 - val_accuracy: 0.1342
Epoch 3/15
704/704 [==============================] - 7s 10ms/step - loss: 1.6963 - accuracy: 0.2492 - val_loss: 1.6882 - val_accuracy: 0.1628
Epoch 4/15
704/704 [==============================] - 7s 10ms/step - loss: 1.4803 - accuracy: 0.3008 - val_loss: 1.4427 - val_accuracy: 0.2680
Epoch 5/15
704/704 [==============================] - 7s 10ms/step - loss: 1.3370 - accuracy: 0.3407 - val_loss: 1.3601 - val_accuracy: 0.3097
Epoch 6/15
704/704 [==============================] - 7s 10ms/step - loss: 1.2753 - accuracy: 0.3580 - val_loss: 1.3066 - val_accuracy: 0.5193
Epoch 7/15
704/704 [==============================] - 7s 10ms/step - loss: 1.2351 - accuracy: 0.3682 - val_loss: 1.2717 - val_accuracy:

In [120]:
for i in range(10):
  ind = np.random.randint(0, len(xval))
  rowx,rowy = xval[np.array([ind])] , yval[np.array([ind])]
  preds = model.predict(rowx)
  pred_maxes = np.argmax(preds, axis=-1)
  knowny = np.argmax(rowy, axis=-1)
  print('****')
  print(str(pred_maxes))
  print(knowny)

****
[[9 8 6 6]]
[[9 8 3 0]]
****
[[2 2 1 6]]
[[2 2 6 0]]
****
[[1 1 2 2]]
[[1 1 2 6]]
****
[[9 8 6 6]]
[[9 8 5 0]]
****
[[1 3 5 5]]
[[1 3 5 5]]
****
[[4 5 6 6]]
[[4 5 5 0]]
****
[[1 5 6 6]]
[[1 5 6 2]]
****
[[5 5 6 6]]
[[5 5 7 0]]
****
[[7 3 6 6]]
[[7 3 5 0]]
****
[[1 7 6 6]]
[[1 8 1 0]]
